In [1]:
### this will follow the example at https://python.langchain.com/docs/expression_language/cookbook/sql_db

In [2]:
%pip install langchain

  Obtaining dependency information for langchain from https://files.pythonhosted.org/packages/f8/f9/66b03ceac5e7ac45392d3ab207e029ec4d13296cff5b04d58319992349c3/langchain-0.0.327-py3-none-any.whl.metadata
  Obtaining dependency information for aiohttp<4.0.0,>=3.8.3 from https://files.pythonhosted.org/packages/91/1b/9b5da3748ddd924288d9a946a0f48a34b179330d342c6cdeb89683bcf971/aiohttp-3.8.6-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
  Obtaining dependency information for anyio<4.0 from https://files.pythonhosted.org/packages/19/24/44299477fe7dcc9cb58d0a57d5a7588d6af2ff403fdd2d47a246c91a3246/anyio-3.7.1-py3-none-any.whl.metadata
  Obtaining dependency information for async-timeout<5.0.0,>=4.0.0 from https://files.pythonhosted.org/packages/a7/fa/e01228c2938de91d47b307831c62ab9e4001e747789d0b05baf779a6488c/async_timeout-4.0.3-py3-none-any.whl.metadata
  Obtaining dependency information for dataclasses-json<0.7,>=0.5.7 from https://files.pythonhosted.org/packages/21/1f

### setup DB on the studio kernel instance

In [15]:
## setup DB
%pwd

'/root/aws-gen-ai/genai-data-sql'

In [16]:
%wget https://raw.githubusercontent.com/lerocha/chinook-database/master/ChinookDatabase/DataSources/Chinook_Sqlite.sql

UsageError: Line magic function `%wget` not found.


In [12]:
from langchain.prompts import ChatPromptTemplate

template = """Human: Based on the table schema below, write a SQL query and just the SQL, nothing else, that would answer the user's question.:
{schema}


Question: {question}
SQL Query:
"""
prompt = ChatPromptTemplate.from_template(template)

In [13]:
from langchain.utilities import SQLDatabase

In [14]:
db = SQLDatabase.from_uri("sqlite:////home/sagemaker-user/chinook.db")


OperationalError: (sqlite3.OperationalError) unable to open database file
(Background on this error at: https://sqlalche.me/e/14/e3q8)

In [7]:
def get_schema(_):
    return db.get_table_info()

In [8]:
def run_query(query):
    return db.run(query)

In [9]:
inference_modifier = {
    "temperature": 1,
    "top_p": .999,
    "top_k": 250,
    "max_tokens_to_sample": 300,
    "stop_sequences": ["\n\nSQL Query:"]
}

In [10]:
from langchain.chat_models import BedrockChat
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import RunnablePassthrough

chat = BedrockChat(model_id="anthropic.claude-v2", model_kwargs=inference_modifier)

# model = ChatOpenAI()

sql_response = (
    RunnablePassthrough.assign(schema=get_schema)
    | prompt
    | chat.bind(stop=["\nSQLResult:"])
    | StrOutputParser()
)

In [11]:
sql_response.invoke({"question": "How many employees are there?"})



NameError: name 'db' is not defined

In [109]:
template = """Based on the table schema below, question, sql query, and sql response, write a natural language response:
{schema}

Question: {question}
SQL Query: {query}
SQL Response: {response}"""
prompt_response = ChatPromptTemplate.from_template(template)

In [110]:
full_chain = (
    RunnablePassthrough.assign(query=sql_response)
    | RunnablePassthrough.assign(
        schema=get_schema,
        response=lambda x: db.run(x["query"]),
    )
    | prompt_response
    | chat
)

In [111]:
full_chain.invoke({"question": "How many employees are there?"})

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/langchain/llms/bedrock.py:50: UserWarning: Error: Prompt must alternate between '

Human:' and '

Assistant:'. Received 

Human: 

Human: Based on the table schema below, write a SQL query and just the SQL, nothing else, that would answer the user's question.:

CREATE TABLE "Album" (
	"AlbumId" INTEGER NOT NULL, 
	"Title" NVARCHAR(160) NOT NULL, 
	"ArtistId" INTEGER NOT NULL, 
	PRIMARY KEY ("AlbumId"), 
	FOREIGN KEY("ArtistId") REFERENCES "Artist" ("ArtistId")
)

/*
3 rows from Album table:
AlbumId	Title	ArtistId
1	For Those About To Rock We Salute You	1
2	Balls to the Wall	2
3	Restless and Wild	2
*/


CREATE TABLE "Artist" (
	"ArtistId" INTEGER NOT NULL, 
	"Name" NVARCHAR(120), 
	PRIMARY KEY ("ArtistId")
)

/*
3 rows from Artist table:
ArtistId	Name
1	AC/DC
2	Accept
3	Aerosmith
*/


CREATE TABLE "Customer" (
	"CustomerId" INTEGER NOT NULL, 
	"FirstName" NVARCHAR(40) NOT NULL, 
	"LastName" NVARCHAR(20) NOT NULL, 
	"Comp

AIMessage(content=' Based on the SQL query and response, there are 8 employees in the Employee table.')